In [52]:
import sys

!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser
!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

sys.path.insert(0, 'usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time

service = Service(executable_path=r'/usr/lib/chromium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 창이 나타나지 않도록 Headless 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# chrome driver
driver = webdriver.Chrome(service=service, options=options)

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/saiarcot895/chromium-beta/ubuntu/ jammy main'
Description:
This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from 

In [53]:
url = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0&t_page=%ED%8F%AC%EB%A7%A8%ED%8A%B8%20%EC%8B%9C%EA%B7%B8%EB%8B%88%EC%B2%98%20%ED%8D%BC%ED%93%B8%2050ml%20(%EC%BD%94%ED%8A%BC%ED%97%88%EA%B7%B8)&t_click=%ED%97%A4%EB%8D%94&t_header_type=%EB%A1%9C%EA%B3%A0'

driver.get(url)

In [66]:
search_for = driver.find_element('xpath', '//*[@id="query"]')
search_for.send_keys('향수')
search_for.send_keys(Keys.ENTER)

time.sleep(2)

perfume = driver.find_element('xpath','//*[@id="w_cate_prd_list"]/li[1]/div/a')
perfume.click()

time.sleep(2)

perfume_review = driver.find_element('xpath','//*[@id="reviewInfo"]/a')
perfume_review.click()
# 1
# '//*[@id="gdasList"]/li[1]/div[1]/div/p/a[1]'
# '//*[@id="gdasList"]/li[1]/div[2]/div[1]/span[1]/span'

# 2
# '//*[@id="gdasList"]/li[2]/div[1]/div/p/a[1]'
# '//*[@id="gdasList"]/li[2]/div[2]/div[1]/span[1]/span'

In [18]:
#print(driver.page_source)

In [68]:
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

data = []
j=0
k = False

while True:
  time.sleep(1)
  i = 1

  while True:
    try:
      # 리뷰어 이름 요소 찾기
      name_xpath = '//*[@id="gdasList"]/li[{}]/div[1]/div/p/a[1]'.format(i)
      review_name_element = driver.find_element('xpath', name_xpath)
      reviewer_name = review_name_element.text

      # 평점 요소 찾기
      rating_xpath = '//*[@id="gdasList"]/li[{}]/div[2]/div[1]/span[1]/span'.format(i)
      review_rating_element = driver.find_element('xpath', rating_xpath)
      reviewer_rating_text = review_rating_element.text

      # 숫자 평점 추출
      match = re.search(r'(\d+)\s*점만점에\s*(\d+)\s*점', reviewer_rating_text)
      if match:
          reviewer_rating = int(match.group(2))
      else:
          reviewer_rating = None

      # 데이터 리스트에 추가
      data.append({'이름': reviewer_name, '평점': reviewer_rating})

      i += 1  # 다음 리뷰로 이동
    except NoSuchElementException:
      break

  try:
    j += 1
    if j==10:
      break
    name_xpath = '//*[@id="gdasContentsArea"]/div/div[7]/a[{}]'.format(j)
    page = driver.find_element('xpath', name_xpath)
    page.click()
  except NoSuchElementException:
    break

df = pd.DataFrame(data)

df

,이름,평점
0,한구슬,5
1,꺄울,5
2,한구슬,5
3,여자향수쓰는아재,1
4,치즈가르마,5
...,...,...
95,그냥그러려니,5
96,tor****,5
97,k202209099****,4
98,clsrn****,5


In [60]:
# 데이터프레임에서 이름 열에 '소진ㅇ'이 있는지 검색
name_to_search = 'cdw'
result = df['이름'].str.contains(name_to_search)

# 결과 출력
if result.any():
    print(f"'{name_to_search}'를 포함한 행이 존재합니다.")
else:
    print(f"'{name_to_search}'를 포함한 행이 존재하지 않습니다.")

'cdw'를 포함한 행이 존재합니다.
